# Building a Simulation

This notebook teaches you how to build a complete simulation using the Simple Autonomous Car SDK.

## What You'll Learn

1. Setting up all components (track, car, sensors, planner, controller, costmap, alerts)
2. Creating a simulation loop
3. Visualizing all components
4. Building custom simulations

## Complete Simulation Architecture

A simulation typically includes:
- **Track**: The racing environment
- **Car**: Vehicle with dynamics
- **Sensors**: Perception (LiDAR, Camera, etc.)
- **Planner**: Path planning
- **Controller**: Control commands
- **Costmap**: Obstacle representation
- **Alerts**: Safety monitoring
- **Visualization**: Real-time display

In [ ]:
import sys

sys.path.insert(0, '../../src')


# Import enhanced visualization for simulations
import sys

import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    Car,
    CarState,
    FrenetMap,
    GridCostmap,
    GroundTruthMap,
    LiDARSensor,
    PerceivedMap,
    PurePursuitController,
    Track,
    TrackBoundsAlert,
    TrackPlanner,
)

sys.path.insert(0, '../../src/simulations')
import enhanced_visualization

## Step 1: Setup All Components

In [ ]:
# 1. Create track
track = Track.create_figure8_track(size=70.0, track_width=6.0, num_points=300)
print(f"✓ Track created: {len(track.centerline)} points")

# 2. Create car
start_point, start_heading = track.get_point_at_distance(0.0)
car = Car(
    initial_state=CarState(
        x=start_point[0],
        y=start_point[1],
        heading=start_heading,
        velocity=12.0
    ),
    wheelbase=2.5,
    max_velocity=25.0,
    max_steering_angle=np.pi / 6,
)
print(f"✓ Car created at ({car.state.x:.2f}, {car.state.y:.2f})")

# 3. Create maps
ground_truth_map = GroundTruthMap(track)
perceived_map = PerceivedMap(
    ground_truth_map,
    position_noise_std=0.15,
    orientation_noise_std=0.08,
    measurement_noise_std=0.25,
)
print("✓ Maps created")

# 4. Create and add sensor
lidar = LiDARSensor(
    ground_truth_map=ground_truth_map,
    perceived_map=perceived_map,
    max_range=40.0,
    angular_resolution=0.1,
    point_noise_std=0.1,
    name="lidar",
)
car.add_sensor(lidar)
print(f"✓ Sensor added: {len(car.sensors)} sensor(s)")

# 5. Create planner
planner = TrackPlanner(track, lookahead_distance=50.0, waypoint_spacing=2.0)
print(f"✓ Planner created: {planner.name}")

# 6. Create controller
controller = PurePursuitController(
    lookahead_distance=12.0,
    lookahead_gain=1.5,
    max_steering_rate=0.8,
    target_velocity=12.0,
    velocity_gain=0.3,
    name="controller",
)
print(f"✓ Controller created: {controller.name}")

# 7. Create costmap
costmap = GridCostmap(
    width=60.0,
    height=60.0,
    resolution=0.5,
    inflation_radius=2.0,
    frame="ego",
)
print(f"✓ Costmap created: {costmap.width}x{costmap.height}m")

# 8. Create alert system
frenet_map = FrenetMap(track)
alert_system = TrackBoundsAlert(
    frenet_map,
    warning_threshold=1.5,
    critical_threshold=3.0,
    lookahead_distance=30.0,
)
print("✓ Alert system created")

print("\n" + "="*60)
print("All components initialized!")
print("="*60)

## Step 2: Visualize Initial Setup

In [ ]:
# Visualize initial state
fig, ax = plt.subplots(figsize=(12, 10))
plot_track(track, ax=ax)
plot_car(car, ax=ax, show_heading=True)
plt.title("Initial Setup")
plt.legend()
plt.show()

## Step 3: Create Simulation Loop

In [ ]:
# Simulation parameters
dt = 0.1
num_steps = 200
horizon = 40.0

# Storage for visualization
trajectory = []
plans_history = []
control_history = []

print("Starting simulation...")
print("="*60)

for step in range(num_steps):
    # 1. Get perception from sensors
    perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})

    # 2. Update costmap
    costmap.update(perception_data, car.state)

    # 3. Generate plan (planner uses perception and costmap)
    plan = planner.plan(car.state, perception_data=perception_data, costmap=costmap)
    plans_history.append(plan.copy() if len(plan) > 0 else None)

    # 4. Compute control (controller uses plan, perception, and costmap)
    control = controller.compute_control(
        car_state=car.state,
        perception_data=perception_data,
        costmap=costmap,
        plan=plan,
        dt=dt,
    )
    control["time"] = step * dt
    control["velocity"] = car.state.velocity
    control_history.append(control.copy())

    # 5. Update car
    car.update(dt, acceleration=control["acceleration"], steering_rate=control["steering_rate"])
    trajectory.append(car.state.position().copy())

    # 6. Check alerts
    perception_points = perception_data.get("lidar")
    if perception_points is not None:
        alert_result = alert_system.check(perception_points, car.state)
        if alert_result["has_critical"]:
            print(f"Step {step:3d}: CRITICAL - Max deviation: {alert_result['max_deviation']:.2f}m")
        elif alert_result["has_warning"]:
            print(f"Step {step:3d}: WARNING - Max deviation: {alert_result['max_deviation']:.2f}m")

print("="*60)
print(f"Simulation complete! Car traveled {len(trajectory)} steps")
print(f"Final position: ({car.state.x:.2f}, {car.state.y:.2f})")
print(f"Final velocity: {car.state.velocity:.2f} m/s")

## Step 4: Visualize Results

In [ ]:
# Get final state for visualization
perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
costmap.update(perception_data, car.state)
plan = planner.plan(car.state, perception_data=perception_data, costmap=costmap)
perception_points = perception_data.get("lidar")

# Use enhanced visualization (World + Ego views with all overlays)
fig, axes = enhanced_visualization.create_enhanced_visualization(figsize=(20, 10))

# Calculate view bounds for world frame
all_points = np.vstack([track.inner_bound, track.outer_bound])
track_x_min, track_x_max = np.min(all_points[:, 0]), np.max(all_points[:, 0])
track_y_min, track_y_max = np.min(all_points[:, 1]), np.max(all_points[:, 1])
padding = 15.0
view_bounds = (track_x_min - padding, track_x_max + padding, track_y_min - padding, track_y_max + padding)

# Update visualization with all components
enhanced_visualization.update_all_views(
    axes=axes,
    track=track,
    car=car,
    plan=plan,
    perception_points=perception_points,
    costmap=costmap,
    controller=controller,
    view_bounds=view_bounds,
    horizon=horizon,
    control_history=control_history,
)

plt.show()

print("\n✓ Enhanced visualization complete!")
print("  - World View: Shows track, costmap, plan, controller overlays, perception, car")
print("  - Ego View: Shows costmap, plan, controller overlays, perception in car frame")

## Step 5: Building a Custom Simulation

You can now build your own simulation by:

1. **Customizing components**: Modify planner, controller, or sensor parameters
2. **Adding custom logic**: Add your own planning or control algorithms
3. **Extending visualization**: Create custom plots for your needs
4. **Adding features**: Integrate new sensors, controllers, or planners

### Example: Custom Simulation Function

In [ ]:
def run_custom_simulation(
    track_type="figure8",
    track_size=70.0,
    initial_velocity=12.0,
    num_steps=200,
    dt=0.1,
    visualize=True,
):
    """
    Run a custom simulation with configurable parameters.

    This is a template you can customize for your needs.
    """
    # Create track
    if track_type == "figure8":
        track = Track.create_figure8_track(size=track_size, track_width=6.0, num_points=300)
    elif track_type == "simple":
        track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0, num_points=200)
    else:
        track = Track.create_oval_track(length=80.0, width=40.0, track_width=6.0, num_points=200)

    # Create all components (same as above)
    start_point, start_heading = track.get_point_at_distance(0.0)
    car = Car(
        initial_state=CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=initial_velocity),
        wheelbase=2.5,
        max_velocity=25.0,
        max_steering_angle=np.pi / 6,
    )

    ground_truth_map = GroundTruthMap(track)
    perceived_map = PerceivedMap(ground_truth_map)
    lidar = LiDARSensor(ground_truth_map, perceived_map, max_range=40.0, name="lidar")
    car.add_sensor(lidar)

    planner = TrackPlanner(track, lookahead_distance=50.0, waypoint_spacing=2.0)
    controller = PurePursuitController(lookahead_distance=12.0, target_velocity=12.0, name="controller")
    costmap = GridCostmap(width=60.0, height=60.0, resolution=0.5, inflation_radius=2.0, frame="ego")

    frenet_map = FrenetMap(track)
    TrackBoundsAlert(frenet_map, warning_threshold=1.5, critical_threshold=3.0)

    # Run simulation loop
    trajectory = []
    for step in range(num_steps):
        perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
        costmap.update(perception_data, car.state)
        plan = planner.plan(car.state, perception_data=perception_data, costmap=costmap)
        control = controller.compute_control(car.state, perception_data=perception_data, costmap=costmap, plan=plan, dt=dt)
        car.update(dt, acceleration=control["acceleration"], steering_rate=control["steering_rate"])
        trajectory.append(car.state.position().copy())

    # Visualize if requested
    if visualize:
        fig, ax = plt.subplots(figsize=(12, 10))
        plot_track(track, ax=ax)
        if len(trajectory) > 1:
            traj_array = np.array(trajectory)
            ax.plot(traj_array[:, 0], traj_array[:, 1], 'b-', linewidth=2, label='Trajectory')
        plot_car(car, ax=ax, show_heading=True)
        ax.set_title(f"Custom Simulation - {track_type} track")
        ax.legend()
        plt.show()

    return car, trajectory

# Example usage
print("Running custom simulation...")
car_result, traj = run_custom_simulation(track_type="figure8", num_steps=150, visualize=True)
print("\n✓ Custom simulation complete!")
print(f"  Final position: ({car_result.state.x:.2f}, {car_result.state.y:.2f})")

## Summary

You've learned how to:

1. ✅ **Setup all components**: Track, Car, Sensors, Planner, Controller, Costmap, Alerts
2. ✅ **Create simulation loop**: Perception → Costmap → Plan → Control → Update
3. ✅ **Visualize everything**: Track, plan, perception, costmap, control history
4. ✅ **Build custom simulations**: Template function for your own simulations

### Key Concepts

- **Modular architecture**: Each component is independent and reusable
- **Data flow**: Sensors → Perception → Costmap → Planner → Controller → Car
- **Visualization**: Use built-in functions for easy plotting
- **Extensibility**: Easy to add custom components

### Next Steps

- Experiment with different track types and parameters
- Try different controllers (PID, custom)
- Build custom planners (A*, RRT)
- Add more sensors (Camera, Radar)
- Create custom alert systems